In [ ]:
"""
Code for preparing data for CMIP6 HighResMIP model simulations, NPD_eORCA025 atmosphere-forced ocean-only 
model runs, and observations-based datasets

Processing in this notebook includes:
-------------------------------------
1. Computing con. temperature data and interpolating/remapping to a regular lat-lon grid 
over the Pacific (30˚S-30˚N; 120˚E-290˚E), using bilinear interpolation. The grid description
for which was manually configured to be a 0.25˚ regular grid and saved as ./data/griddes_025.grd

2. Calculating anomalies using centred moving 30yr baselines, for which the bounds were
manually configured and saved as a .csv files in ./data/ (with prefix as Centred_BS_bounds),
separetely for CMIP6 HighResMIP total period (1950-2050) and Observational datasets and NPD_eOR025 
runs (1976-2023).

3. Calculate depth of 20˚ isotherm from computed con. temp data, and saving the file.

4. Compute zonal wind anomalies at 850hPa for CMIP6 HighResMIP and ERA5 Reanalysis.

-------------------------------------------------------------------------------------------
Author: Sreevathsa G. (sg13n23@soton.ac.uk; ORCID ID: 0000-0003-4084-9677)
Last updated: 11 December 2025
"""

In [ ]:
import gc
import sys
import gsw
import glob
import xarray as xr
import os
from itertools import product
from utils import setup_dask_cluster, cdo_remapbil, z20_calculator, flatten, renamer
from tqdm.notebook import tqdm

In [ ]:
# Dataset name lists
PARENT_DIR = '/badc/cmip6/data/CMIP6/HighResMIP/'
HIGHRESMIP_MODEL_PATH_SUFFIX = ['MPI-ESM1-2-HR', 'MPI-ESM1-2-XR', 'HadGEM3-GC31-HH', 
                                'EC-Earth3P-HR', 'CMCC-CM2-VHR4', 'CNRM-CM6-1-HR']
EXP_TYPES = {'CTRL' : ['control-1950'], 
             'HIST-FUT' : ['hist-1950', 'highres-future']}

In [ ]:
# Setting up the Dask client for this notebook.
cluster, client = setup_dask_cluster()
client

In [ ]:
# =============================================================================
# CALCULATING CON. TEMPERATURE DATA OVER A PACIFIC SUBSET USING A GSW FUNCTION
# =============================================================================

# CMIP6 HighResMIP
# ----------------

for model, exp_type in tqdm(list(product(HIGHRESMIP_MODEL_PATH_SUFFIX, EXP_TYPES))):
    # Creating a {model}_{exp_type} sub-directory if it doesn't exist
    if os.path.exists(f'./data/thetao_con/{model}_{exp_type}/') == False:
        os.mkdir(f'./data/thetao_con/{model}_{exp_type}/')
    for year in tqdm(np.arange(1950, 2050+1), leave = False):
        # Filenames corresponding to each year's data: pt = potential temperature (var_name : thetao, in degC) and so = absolute salinity (var_name : so, in g/kg)
        fnames_pt = flatten([sorted(glob.glob(PARENT_DIR + f'*/{model}/{exp_id}/r1*/Omon/thetao/*/latest/*{year}??.nc')) for exp_id in EXP_TYPES[exp_type]])
        fnames_so = flatten([sorted(glob.glob(PARENT_DIR + f'*/{model}/{exp_id}/r1*/Omon/so/*/latest/*{year}??.nc')) for exp_id in EXP_TYPES[exp_type]])
        # Ensuring the lists are not empty i.e., files for the iterated year exists
        if (fnames_pt != []) & (fnames_so != []):
            # Reading and bilinearly interpolating the data into a 0.25deg regular lat-lon grid
            ds_thetao = cdo_remapbil(filename_list = fnames_pt, var_name = 'thetao')
            ds_so = cdo_remapbil(filename_list = fnames_so, var_name = 'so')
            # Computing conservative temperature from potential temperature and salinity using gsw.conversions.CT_from_pt function
            ds = gsw.conversions.CT_from_pt(SA = ds_so['so'], pt = ds_thetao['thetao']).rename('thetao_con').to_dataset()
            # Saving the file as a single-year file
            ds.to_netcdf(f'./data/thetao_con/{model}_{exp_type}/BILINTERP_PAC_THETAO_CON_CMIP6_HIGHRESMIP_{model}_{exp_type}_{year}.nc')
    # Cleaning-up cdo tempdir
    cdo.cleanTempDir()
    
# Observations-based Products
# ---------------------------

for obs_dataset in ['ORAS5', 'EN4']:
    # The original files are saved at per-month frequency, so looping through each monthly timestep from 1976-2023
    for ts in tqdm(pd.date_range(start = '1976-01-01', end = '2023-12-31', freq = '1MS')):
        # Creating a {obs_dataset} sub-directory if it doesn't exist
        if os.path.exists(f'./data/thetao_con/{obs_dataset}/') == False:
            os.mkdir(f'./data/thetao_con/{obs_dataset}/')

        # Filenames corresponding to each year's data: pt = potential temperatur and so = absolute salinity, varname_pt/so set accordingly
        if obs_dataset == 'ORAS5':
            f_so = glob.glob(f'/dssgfs01/working/sg13n23/OBS_DATA/ORAS5/orig_data/vosaline*{ts.year}{ts.month:02d}*')[0]
            f_pt = glob.glob(f'/dssgfs01/working/sg13n23/OBS_DATA/ORAS5/orig_data/votemper*{ts.year}{ts.month:02d}*')[0]
            varname_pt = 'votemper'
            varname_so = 'vosaline'
        elif obs_dataset == 'EN4':
            f_so = f_pt = f'/dssgfs01/working/sg13n23/OBS_DATA/EN4/orig_data/EN.4.2.2.f.analysis.g10.{ts.year}{ts.month:02d}.nc'
            varname_pt = 'temperature'
            varname_so = 'salinity'
            
        # Reading and bilinearly interpolating the data into a 0.25deg regular lat-lon grid
        ds_thetao = cdo_remapbil(filename_list = [f_pt], var_name = varname_pt)
        ds_so = cdo_remapbil(filename_list = [f_so], var_name = varname_so)
        # Computing con. temperature from potential temperature and salinity using gsw.conversions.CT_from_pt function
        ds = gsw.conversions.CT_from_pt(SA = ds_so[varname_so], pt = ds_thetao[varname_pt]).rename('thetao_con').to_dataset()
        # Setting time axis so that it's the start of the month
        ds['time'] = pd.to_datetime([str(_)[0:8]+'01' for _ in ds['time'].values])
        # Saving the file as a single-year file
        ds.to_netcdf(f'./data/thetao_con/{obs_dataset}/BILINTERP_PAC_THETAO_CON_{obs_dataset}_{ts.year}{ts.month:02d}.nc')
        # Cleaning-up cdo tempdir
        cdo.cleanTempDir()  

# NPD_eORCA025 experiments 
# -------------------------
# NOTE: thetao_con is pre-calculated online when the model is run, so just bilinearly interpolating and saving data here

for npd_exp in ['NPD_eORCA025_ERA5', 'NPD_eORCA025_JRA55']:
    # The original files are saved at per-month frequency, so looping through each monthly timestep from 1976-2023
    for ts in tqdm(pd.date_range(start = '1976-01-01', end = '2023-12-31', freq = '1MS')):
        # Reading and bilinearly interpolating the data into a 0.25deg regular lat-lon grid
        fname = f'/dssgfs01/scratch/npd/simulations/{npd_exp[4:]}/{ts.year}/eORCA025_1m_grid_T_{ts.year}{ts.month:02d}-{ts.year}{ts.month:02d}.nc'
        ds = cdo_remapbil(filename_list = [fname], var_name = 'thetao_con')['thetao_con'].to_dataset()
        # Setting time axis appropriately
        ds['time'] = pd.to_datetime([str(_)[0:8]+'01' for _ in ds['time'].values])
        # Setting time axis so that it's the start of the month
        ds.to_netcdf(f'./data/thetao_con/{npd_exp}/BILINTERP_PAC_THETAO_CON_{npd_exp}_{ts.year}{ts.month:02d}.nc')
        # Cleaning-up cdo tempdir
        cdo.cleanTempDir() 

In [ ]:
# =============================================================================
# CALCULATING MOVING-BASELINE ANOMALIES FOR EACH DATASET OVER THE PACIFIC
# =============================================================================

# CMIP6 HighResMIP
# ----------------

# Reading in the CSV file containing the 30-year climatological (centred) baseline bounds for consecutive 5 years in 1950-2050 (file prepared manually)
bounds_df = pd.read_csv('./data/Centred_BS_bounds_CMIP6_HIGHRESMIP_1950-2050.csv')
for model, exp_type in tqdm(list(product(HIGHRESMIP_MODEL_PATH_SUFFIX, EXP_TYPES))):
    # Creating a {model}_{exp_type} directory if it doesn't exist
    if os.path.exists(f'./data/anomalies/{model}_{exp_type}/') == False:
        os.mkdir(f'./data/anomalies/{model}_{exp_type}/')

    # Lazy-loading (Dask) in all the con. temp. data (thetao_con) processed and saved above, using open_mfdataset
    ds = xr.open_mfdataset(f'./data/thetao_con/{model}_{exp_type}/BILINTERP_PAC_THETAO_CON_CMIP6_HIGHRESMIP_{model}_{exp_type}_????.nc')
    # Setting time axis so that it's the start of the month, and is of datetime64[ns] type
    ds['time'] = pd.date_range(start = '1950-01-01', periods = ds.time.shape[0], freq = '1MS')

    # Looping through each year from 1950 to 2050, to calculate each year's anomalies
    for year in tqdm(np.arange(1950, 2050+1), leave = False):
        # Subsetting the row in bounds_df dataframe, where 'year' lies in (START_Y, END_Y) (i.e., the 5-year period), to determine the baseline bounds to use
        bs_df = bounds_df[(year>=bounds_df['START_Y']) & (year<=bounds_df['END_Y'])]
        bs_start, bs_end = str(bs_df['BS_START'].item()), str(bs_df['BS_END'].item())

        # Calculating monthly mean climatological baseline, corresponding to the 5-year period where 'year' lies in
        baseline = ds.sel(time = slice(bs_start, bs_end)).groupby('time.month').mean()
        # Calculating the temperature anomalies, and renaming the variable appropriately
        anomalies = ds.sel(time = str(year)).groupby('time.month') - baseline
        anomalies = anomalies.rename({'thetao_con':'anomaly'}).drop_vars('month')
        # Setting attribute information according to the processing done
        anomalies.attrs['standard_name'] = 'anomaly_sea_water_conservative_temperature'
        anomalies.attrs['long_name'] = f'Anomaly of Sea Water Conservative Temperature with baseline {bs_start}-{bs_end}'
        anomalies.attrs['units'] = 'degC'
        # Saving the processed anomaly data
        anomalies.to_netcdf(f'./data/anomalies/{model}_{exp_type}/BILINTERP_PAC_ANOM_BS{bs_start}-{bs_end}_CMIP6_HIGHRESMIP_{model}_{exp_type}_{year}.nc')

# Observations-based Products and NPD_eORCA025 exps.
# --------------------------------------------------

# Reading in the CSV file containing the 30-year climatological (centred) baseline bounds for consecutive 5 years in 1976-2023 recent past period (file prepared manually)
bounds_df = pd.read_csv('./data/Centred_BS_bounds_NPD_eORCA025_OBS_1976-2023.csv')
for dataset in ['ORAS5', 'EN4', 'NPD_eORCA025_ERA5', 'NPD_eORCA025_JRA55']:
    # Creating a {dataset} sub-directory if it doesn't exist
    if os.path.exists(f'./data/anomalies/{dataset}/') == False:
            os.mkdir(f'./data/anomalies/{dataset}/')
    # Looping through each monthly timestamp, as done previously while preparing and saving thetao_con for observations and NPD experiments
    for ts in tqdm(pd.date_range(start = '1976-01-01', end = '2023-12-31', freq = '1MS')):
        # Subsetting the row in bounds_df dataframe, where 'year' lies in (START_Y, END_Y) (i.e., the 5-year period), to determine the baseline bounds to use
        bs_df = bounds_df[(ts.year>=bounds_df['START_Y']) & (ts.year<=bounds_df['END_Y'])]
        bs_start, bs_end = bs_df['BS_START'].item(), bs_df['BS_END'].item()

        # Making a list of files to load to compute the climatological baseline corresponding to 'ts.month'
        bs_fnames = []
        for _ in range(bs_start, bs_end+1):
            bs_fnames += [f'./data/thetao_con/{dataset}/BILINTERP_PAC_THETAO_CON_{dataset}_{_}{ts.month:02d}.nc']
            
        # Loading in the data file for 'ts', computing climatological baseline mean and computing anomalies
        ds = xr.open_dataset(f'./data/thetao_con/{dataset}/BILINTERP_PAC_THETAO_CON_{dataset}_{ts.year}{ts.month:02d}.nc')
        baseline = xr.open_mfdataset(bs_fnames).groupby('time.month').mean()
        anomalies = ds.groupby('time.month') - baseline
        # Renaming the variable appropriately
        anomalies = anomalies.rename({'thetao_con':'anomaly'}).drop_vars('month')
        # Setting attribute information according to the processing done
        anomalies.attrs['standard_name'] = 'anomaly_sea_water_conservative_temperature'
        anomalies.attrs['long_name'] = f'Anomaly of Sea Water Conservative Temperature with baseline {bs_start}-{bs_end}'
        anomalies.attrs['units'] = 'degC'
        # Saving the processed anomaly data
        anomalies.to_netcdf(f'./data/anomalies/{dataset}/BILINTERP_PAC_ANOM_BS{bs_start}-{bs_end}_{dataset}_{ts.year}{ts.month:02d}.nc')

In [ ]:
# =============================================================================
# CALCULATING DEPTH OF 20°C (Z20) FOR EACH MODEL/OBSERVATIONAL DATASET
# =============================================================================

# CMIP6 HighResMIP
# ----------------

for model, exp_type in tqdm(list(product(HIGHRESMIP_MODEL_PATH_SUFFIX, EXP_TYPES))):
    ds_z20 = [] # Empty list to save computed Z20 for each year
    # Looping through each year from 1950 to 2050
    for year in tqdm(np.arange(1950, 2050+1), leave = False):
        # Loading in the data and ensuring that the time axis is the start of the month, and is of datetime64[ns] type
        ds = xr.open_dataset(f'./data/thetao_con/{model}_{exp_type}/BILINTERP_PAC_THETAO_CON_CMIP6_HIGHRESMIP_{model}_{exp_type}_{year}.nc')
        ds['time'] = pd.date_range(start = f'{year}-01-01', periods = ds.time.shape[0], freq = '1MS')
        # Cropping to 5˚S - 5˚N
        ds = ds.sel(lat = slice(-5,5), lon = slice(140,280))
        # Calculating Z20 depths and appending to the ds_z20 list
        ds_z20 += [z20_calculator(ds)]
    # Merging calculated Z20 for all years to a single dataset
    ds_z20 = xr.merge(ds_z20)
    # Interpolating to a finer longitude grid, averaging across 'lat' dimension and expanding dims to include 'model' name information
    ds_z20 = ds_z20.interp(lon = np.arange(140,280.01, 0.1)).mean(dim = ['lat']).expand_dims(dataset = [model])
    # Saving the processed Z20 data
    ds_z20.to_netcdf(f'./data/z20_depths/Z20_DEPTHS_{model}_{exp_type}_1950-2050.nc')

# Observations-based Products and NPD_eORCA025 exps.
# --------------------------------------------------

for dataset in ['ORAS5', 'EN4', 'NPD_eORCA025_ERA5', 'NPD_eORCA025_JRA55']:
    ds_z20 = [] # Empty list to save computed Z20 for each monthly timestep
    # Looping through each monthly timestep (as files are saved as such) from 1976 to 2023 of the recent past period
    for ts in tqdm(pd.date_range(start = '1976-01-01', end = '2023-12-31', freq = '1MS')):
        # Loading in the data and ensuring that the time axis is the start of the month, and is of datetime64[ns] type
        ds = xr.open_dataset(f'./data/thetao_con/{dataset}/BILINTERP_PAC_THETAO_CON_{dataset}_{ts.year}{ts.month:02d}.nc')
        ds['time'] = pd.date_range(start = f'{year}-01-01', periods = ds.time.shape[0], freq = '1MS')
        # Cropping to 5˚S - 5˚N
        ds = ds.sel(lat = slice(-5,5), lon = slice(140,280))
        # Calculating Z20 depths and appending to the ds_z20 list
        ds_z20 += [z20_calculator(ds)]
    # Merging calculated Z20 for all monthly timesteps to a single dataset
    ds_z20 = xr.merge(ds_z20)
    # Interpolating to a finer longitude grid, averaging across 'lat' dimension and expanding dims to include 'dataset' name information
    ds_z20 = ds_z20.interp(lon = np.arange(140,280.01, 0.1)).mean(dim = ['lat']).expand_dims(dataset = [dataset])
    # Saving the processed Z20 data
    ds_z20.to_netcdf(f'./data/z20_depths/Z20_DEPTHS_{dataset}_1950-2050.nc')

In [ ]:
# =============================================================================
# CALCULATING ZONAL WIND ANOMALIES FOR CMIP6 HIGHRESMIP AND ERA5 REANALYSIS
# =============================================================================

# CMIP6 HighResMIP
# ----------------

for model, exp_type in tqdm(list(product(HIGHRESMIP_MODEL_PATH_SUFFIX, EXP_TYPES))):
    # Zonal Wind @ 850 hPa - Bilinearly interpolated to a subset over the Pacific
    # ---------------------------------------------------------------------------
    
    # Reading and bilinearly interpolating the data into a 0.25deg regular lat-lon grid, selecting a single pressure level of interest (argument passed in hPa)
    fnames_ua = flatten([sorted(glob.glob(PARENT_DIR + f'*/{model}/{exp_id}/r1*/Amon/ua/*/latest/*.nc')) for exp_id in EXP_TYPES[exp_type]])
    ds = cdo_remapbil(filename_list = fnames_ua, var_name = 'ua', plev = 850).squeeze()
    # Ensuring that the time axis is the start of the month, and is of datetime64[ns] type
    ds['time'] = pd.date_range(start = '1950-01-01', periods = ds.time.shape[0], freq = '1MS')
    # Saving the interpolated/cropped zonal wind data @ 850 hPa
    ds.to_netcdf(f'./data/zon_wnds_850hpa/BILINTERP_PAC_ZON_WND_850HPA_{model}_{exp_type}.nc')
    # Cleaning-up cdo tempdir
    cdo.cleanTempDir()
    
    # Calculating zonal wind anomalies @ 850 hPa
    # ------------------------------------------
    
    # Compute anomalies separately for each forcing group, using a full-period
    # climatology specific to each sub-period:
    #   - ctrl-1950      : mean over 1950–2050
    #   - hist-1950      : mean over 1950–2014
    #   - highres-future : mean over 2015–2050
    # In other words, each experiment's anomalies are computed relative to its
    # own full available climatology, not a shared baseline across groups.

    if exp_type == 'CTRL':
        ds_anomalies = ds['ua'].groupby('time.month') - ds['ua'].groupby('time.month').mean()
    elif exp_type == 'HIST-FUT':
        ds_anomalies = xr.concat([ds['ua'].sel(time = slice('1950', '2014')).groupby('time.month') 
                                  - ds['ua'].sel(time = slice('1950', '2014')).groupby('time.month').mean(),
                                  
                                  ds['ua'].sel(time = slice('2015', '2050')).groupby('time.month') 
                                  - ds['ua'].sel(time = slice('2015', '2050')).groupby('time.month').mean()],
                                dim = 'time')
    
    # Saving information on the climatological periods used to calculate anomalies in attrs.history, for future reference
    ds_anomalies.attrs['history'] = 'Zonal wind anomalies calculated with respect to climatological mean over periods: 1950-2050 (control-1950), 1950-2014 (hist-1950) and 2015-2050 (highres-future).'
    # Saving the processed zonal wind anomalies @ 850 hPa
    ds_anomalies.to_netcdf(f'./data/zon_wnds_850hpa/BILINTERP_PAC_ZON_WND_ANOM_850HPA_{model}_{exp_type}.nc')

# ERA5 Reanalysis
# ----------------
# Reading and bilinearly interpolating the data into a 0.25deg regular lat-lon grid
ds_era5 = cdo.remapbil('./data/griddes_025.grd', input='./data/zon_wnds_850hpa/UV_WND_850HPA_ERA5_REANALYSIS.nc', returnXDataset = True)
# Saving the cropped/interpolated zonal wind @ 850 hPa data
ds_era5.to_netcdf('./data/zon_wnds_850hpa/BILINTERP_PAC_UV_WND_850HPA_ERA5_REANALYSIS.nc')

# Selecting data for only 1976-2023 recent past period, renaming some variables, and squeezing 1-value dimensions
ds_era5 = ds_era5.rename({'valid_time': 'time', 'u':'ua', 'v':'va'}).sel(time = slice('1976', '2023')).squeeze()
# calculating zonal wind anomalies using the full 1976-2023 period as the climatological baseline
ds_era5_anoms = ds_era5.groupby('time.month') - ds_era5.groupby('time.month').mean()
# Saving the processed zonal wind anomalies @ 850 hPa
ds_era5_anoms.to_netcdf('./data/zon_wnds_850hpa/BILINTERP_PAC_UV_WND_ANOM_850HPA_ERA5_REANALYSIS_1976-2023.nc')

# NOTE: The file ./data/zon_wnds_850hpa/UV_WND_850HPA_ERA5_REANALYSIS.nc was obtained directly from the Copernicus Climate Data Store 
# (ERA5, 850 hPa u and v winds). The two wind components were downloaded and merged into a single NetCDF file externally; this step is 
# not part of the preprocessing pipeline implemented here.

In [ ]:
client.shutdown()
cluster.close()